In [1]:
import glob
import pandas as pd 

#map renewables

df_renew = pd.concat(map(pd.read_csv, glob.glob('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/raw_data_renewable/*.csv')))
df_renew.rename(columns={'Country or Area':'Geo'}, inplace=True)

#map nonrenewables

df_nonrenew = pd.concat(map(pd.read_csv, glob.glob('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/raw_data_nonrenewable/*.csv')))
df_nonrenew.rename(columns={'Country or Area':'Geo'}, inplace=True)

df = pd.concat([df_renew, df_nonrenew])

#split commodity and transaction into two columns
#defining column to split and which delimiter
ct = df['Commodity - Transaction'].str.split('-')

#assigning new columns
df['Commodity'] = ct.str.get(0)
df['Transaction'] = ct.str.get(1)

#lowercase and remove trailing and leading spaces
df['Commodity'] = df['Commodity'].str.lower()
df['Commodity'] = df['Commodity'].str.strip()

df['Transaction'] = df['Transaction'].str.lower()
df['Transaction'] = df['Transaction'].str.strip()

df = df[df['Transaction'] == 'production']

In [7]:
import pandas as pd 
unit_conv = pd.read_csv('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/renewable_energy_units_conv_2.csv')

df_merge = pd.merge(df,  
                     unit_conv,  
                     on ='Commodity', 
                     how ='left') 

df_merge['TJ'] = df_merge['Quantity'] * df_merge['Conv_mt_to_tj']
df_merge.sample(3)



,Geo,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes,Commodity,Transaction,Description,Units,Conv_mt_to_tj,r_type,TJ
36069,Russian Federation,Brown coal - production,2000.0,"Metric tons, thousand",87786.0,NaN,brown coal,production,"Lignite, compressed peat","Metric tons, thousand",14.00,nonrenewable,1229004.0
33584,Switzerland,Bitumen - Production,1993.0,"Metric tons, thousand",119.0,NaN,bitumen,production,"Asphalt, also known as bitumen, is a sticky, b...","Metric tons, thousand",40.20,nonrenewable,4783.8
14509,Rwanda,Fuelwood - Production,1993.0,"Cubic metres, thousand",3000.0,NaN,fuelwood,production,Wood used as fuel,"Cubic meters, thousand",9.14,renewable,27420.0


In [9]:
lt_2018 = df_merge[df_merge['Year'] == 2018]
#greate geo group
tot_grp_country = lt_2018.groupby(['Geo'])
#sum TJ by geo group
df_tot_country = tot_grp_country['TJ'].agg(['sum'])

df_tot_country.to_csv('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/countries_from_energy_sources.csv')